In [11]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import os
import threading
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


# 🌿 Define labels and image size
class_names = ['Healthy', 'Leaf Spot', 'Rust']
image_size = (224, 224)

# 🌱 Create and train model
def create_model(input_shape=(224, 224, 3), num_classes=4):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def train_model():
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_gen = datagen.flow_from_directory(
        'dataset/',
        target_size=image_size,
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )
    val_gen = datagen.flow_from_directory(
        'dataset/',
        target_size=image_size,
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )
    model = create_model(num_classes=len(train_gen.class_indices))
    model.fit(train_gen, epochs=10, validation_data=val_gen)
    model.save(model_path)
    val_loss, val_acc = model.evaluate(val_gen)
    print(f"✅ Validation Accuracy: {val_acc * 100:.2f}%")
    return model

# 🧠 Load or train the model
if os.path.exists(model_path):
    model = load_model(model_path)
else:
    print("Training now...")
    model = train_model()

# 🌼 Prediction function
def predict_disease(img_path):
    try:
        img = image.load_img(img_path, target_size=image_size)
        img_array = image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        prediction = model.predict(img_array)[0]
        predicted_index = np.argmax(prediction)
        predicted_class = class_names[predicted_index]
        confidence = prediction[predicted_index] * 100

        return f"{predicted_class} ({confidence:.2f}% confidence)"
    except Exception as e:
        return f"Error during prediction: {str(e)}"

# 🖼️ GUI upload + display
def upload_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png *.jpg *.jpeg *.bmp")])
    if file_path:
        try:
            img = Image.open(file_path)
            img.thumbnail((400, 400))
            img_tk = ImageTk.PhotoImage(img)
            panel.config(image=img_tk)
            panel.image = img_tk

            result_label.config(text="🔍 Analyzing...")
            root.update()

            def predict_and_update():
                result = predict_disease(file_path)
                result_label.config(text=f"🧠 Prediction:\n{result}")

            threading.Thread(target=predict_and_update).start()

        except Exception as e:
            messagebox.showerror("Error", f"Could not process image:\n{e}")

# 🪟 GUI Design
root = tk.Tk()
root.title("🌿 Smart Leaf Disease Detector")
root.geometry("600x700")
root.configure(bg="#f0f9f2")

title = tk.Label(root, text="🌱 Upload a Leaf Image", font=("Helvetica", 22, "bold"), bg="#f0f9f2", fg="#2e7d32")
title.pack(pady=20)

upload_btn = tk.Button(root, text="📁 Choose Image", command=upload_image,
                       font=("Helvetica", 14), bg="#66bb6a", fg="white", padx=10, pady=5)
upload_btn.pack(pady=10)

panel = tk.Label(root, bg="#f0f9f2")
panel.pack(pady=20)

result_label = tk.Label(root, text="🧠 Prediction: ", font=("Helvetica", 16), bg="#f0f9f2", fg="#1b5e20")
result_label.pack(pady=20)

footer = tk.Label(root, text="© 2025 PlantAI - All rights reserved", font=("Arial", 10), bg="#f0f9f2", fg="#9e9e9e")
footer.pack(side="bottom", pady=10)

root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
